# MFEGSN — Pipeline Colab (Marker + LangExtract)

Notebook optimisé pour une exécution **pas à pas** sur Google Colab.

**🚀 Optimisation:** Les modèles (~4GB) sont sauvegardés sur Google Drive après le premier téléchargement.
Les sessions suivantes chargeront les modèles depuis Drive en quelques secondes.

**Ordre recommandé:** Étapes 1 → 10

## Étape 1 — Monter Google Drive
**IMPORTANT:** Exécutez cette cellule EN PREMIER pour permettre le cache des modèles.

In [1]:
from google.colab import drive
drive.mount("/content/drive")

# Créer le dossier de cache sur Drive
import os
from pathlib import Path

# === CONFIGURATION DU CACHE ===
DRIVE_CACHE_DIR = Path("/content/drive/MyDrive/.mfegsn_cache")
DRIVE_CACHE_DIR.mkdir(parents=True, exist_ok=True)

# Sous-dossiers pour chaque type de cache
HF_CACHE_DRIVE = DRIVE_CACHE_DIR / "huggingface"
TORCH_CACHE_DRIVE = DRIVE_CACHE_DIR / "torch"
DATALAB_CACHE_DRIVE = DRIVE_CACHE_DIR / "datalab"

for cache_dir in [HF_CACHE_DRIVE, TORCH_CACHE_DRIVE, DATALAB_CACHE_DRIVE]:
    cache_dir.mkdir(parents=True, exist_ok=True)

# Configurer les variables d'environnement pour utiliser le cache Drive
os. environ["HF_HOME"] = str(HF_CACHE_DRIVE)
os.environ["TRANSFORMERS_CACHE"] = str(HF_CACHE_DRIVE)
os.environ["HUGGINGFACE_HUB_CACHE"] = str(HF_CACHE_DRIVE)
os.environ["TORCH_HOME"] = str(TORCH_CACHE_DRIVE)
os.environ["XDG_CACHE_HOME"] = str(DRIVE_CACHE_DIR)

# Vérifier la taille du cache existant
def get_dir_size(path):
    total = 0
    if path.exists():
        for f in path.rglob("*"):
            if f.is_file():
                total += f.stat().st_size
    return total / 1e9

cache_size = get_dir_size(DRIVE_CACHE_DIR)

print("="*60)
print("✅ Google Drive monté! ")
print("="*60)
print(f"📂 Cache Drive:  {DRIVE_CACHE_DIR}")
print(f"💾 Taille du cache: {cache_size:.2f} GB")
if cache_size > 3:
    print("🚀 Modèles déjà en cache!  Le chargement sera rapide.")
else:
    print("📥 Premier lancement:  les modèles seront téléchargés et mis en cache.")
print("="*60)

Mounted at /content/drive
✅ Google Drive monté! 
📂 Cache Drive:  /content/drive/MyDrive/.mfegsn_cache
💾 Taille du cache: 3.53 GB
🚀 Modèles déjà en cache!  Le chargement sera rapide.


## Étape 2 — Installer les dépendances
Installation des packages Python nécessaires.

In [2]:
# Dépendances système
!apt-get update -qq
! apt-get install -y zstd -qq

# Dépendances Python
!python -m pip install -q --upgrade pip
!python -m pip install -q marker-pdf[full] langextract google-generativeai pillow

# Installer hf_transfer pour des téléchargements plus rapides (sans casser les dépendances)
!pip install -q hf_transfer --no-deps

# Activer hf_transfer
import os
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

# Vérifier les versions
print("\n📦 Versions installées:")
!pip show huggingface_hub 2>/dev/null | grep Version
! pip show transformers 2>/dev/null | grep Version
!pip show marker-pdf 2>/dev/null | grep Version

print("\n✅ Dépendances installées.")

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package zstd.
(Reading database ... 121689 files and directories currently installed.)
Preparing to unpack .../zstd_1.4.8+dfsg-3build1_amd64.deb ...
Unpacking zstd (1.4.8+dfsg-3build1) ...
Setting up zstd (1.4.8+dfsg-3build1) ...
Processing triggers for man-db (2.10.2-1) ...

📦 Versions installées:
Version: 0.36.0
Version: 4.57.6
Version: 1.10.1

✅ Dépendances installées.


## Étape 3 — Configurer les dossiers de travail
Modifiez les chemins selon votre Drive.

In [3]:
from pathlib import Path

# === MODIFIEZ CES CHEMINS ===
INPUT_DIR = Path("/content/drive/MyDrive/Géopolitique et Souveraineté Numériques/ALL/ALLPDF")
OUTPUT_DIR = Path("/content/drive/MyDrive/Géopolitique et Souveraineté Numériques/ALL/ALLMD")

assert INPUT_DIR.exists(), f"❌ Dossier introuvable: {INPUT_DIR}"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Dossier global pour les logs du pipeline
GLOBAL_LOGS_DIR = OUTPUT_DIR / "_PIPELINE_LOGS"
GLOBAL_LOGS_DIR.mkdir(parents=True, exist_ok=True)

def get_doc_folders(doc_name):
    """Crée et retourne les dossiers pour un document donné."""
    doc_dir = OUTPUT_DIR / doc_name
    doc_dir.mkdir(parents=True, exist_ok=True)

    folders = {
        "root": doc_dir,
        "figures": doc_dir / "_FIGURES",
        "references": doc_dir / "_REFERENCES",
        "analyses": doc_dir / "_ANALYSES",
        "logs": doc_dir / "_LOGS",
    }

    for folder in folders.values():
        folder.mkdir(parents=True, exist_ok=True)

    return folders

pdf_files = sorted([p for p in INPUT_DIR.iterdir() if p.suffix.lower() == ".pdf"])
print("="*60)
print("📁 CONFIGURATION")
print("="*60)
print(f"📂 Entrée: {INPUT_DIR}")
print(f"📂 Sortie: {OUTPUT_DIR}")
print(f"📄 PDFs: {len(pdf_files)}")
print(f"📂 Structure par fichier:  <nom_fichier>/")
print(f"   ├── _ANALYSES/")
print(f"   ├── _FIGURES/")
print(f"   ├── _LOGS/")
print(f"   └── _REFERENCES/")
print("="*60)

📁 CONFIGURATION
📂 Entrée: /content/drive/MyDrive/Géopolitique et Souveraineté Numériques/ALL/ALLPDF
📂 Sortie: /content/drive/MyDrive/Géopolitique et Souveraineté Numériques/ALL/ALLMD
📄 PDFs: 54
📂 Structure par fichier:  <nom_fichier>/
   ├── _ANALYSES/
   ├── _FIGURES/
   ├── _LOGS/
   └── _REFERENCES/


## Étape 4 — Ollama + Gemma 3 4B
Installation et démarrage de Gemma 3 4B via Ollama.

In [4]:
USE_OLLAMA = True  # Activé par défaut pour Gemma local

ollama_process = None

if USE_OLLAMA:
    import subprocess
    import time
    import os

    # Installer requests si nécessaire
    try:
        import requests
    except ImportError:
        !pip install -q requests
        import requests

    # Installer Ollama
    print("📦 Installation d'Ollama...")
    !curl -fsSL https://ollama.com/install.sh 2>/dev/null | sh 2>&1 | tail -n 3

    # Démarrer Ollama en arrière-plan
    ollama_process = subprocess.Popen(
        ["ollama", "serve"],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL
    )
    print("⏳ Démarrage du serveur Ollama...")

    # Attendre que le serveur soit prêt (avec timeout de 30 secondes)
    server_ready = False
    for i in range(30):
        try:
            response = requests.get("http://localhost:11434/api/tags", timeout=2)
            if response.status_code == 200:
                print("✅ Serveur Ollama prêt!")
                server_ready = True
                break
        except:
            pass
        time.sleep(1)

    if not server_ready:
        print("⚠️ Le serveur Ollama n'a pas démarré à temps")
        print("   Essayez de réexécuter cette cellule.")

    # Télécharger Gemma 3 4B (SANS espace dans le nom!)
    print("\n📥 Téléchargement de Gemma 3 4B (≈3GB)...")
    print("   Cela peut prendre plusieurs minutes...")
    !ollama pull gemma3:4b

    # Vérification
    result = subprocess.run(["ollama", "list"], capture_output=True, text=True)
    if "gemma3:4b" in result.stdout or "gemma3" in result.stdout:
        print("\n✅ Gemma 3 4B téléchargé!")

        # Préchauffer le modèle avec une requête test
        print("🔥 Préchauffage du modèle...")
        test_result = subprocess.run(
            ["ollama", "run", "gemma3:4b", "Réponds uniquement: OK"],
            capture_output=True,
            text=True,
            timeout=30
        )

        if test_result.returncode == 0:
            print("✅ Gemma 3 4B prêt et opérationnel!")
            print("\n" + "="*60)
            print("🤖 OLLAMA CONFIGURÉ")
            print("="*60)
            print("Modèle: gemma3:4b")
            print("Serveur: http://localhost:11434")
            print("="*60)
        else:
            print("⚠️ Avertissement: Le test du modèle a échoué")
            print("   Le modèle devrait quand même fonctionner.")
    else:
        print("❌ Erreur: Gemma 3 4B non trouvé")
        print("Modèles disponibles:")
        print(result.stdout if result.stdout else "Aucun modèle")
else:
    print("ℹ️ Ollama désactivé. Mettez USE_OLLAMA = True pour l'activer.")

📦 Installation d'Ollama...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
⏳ Démarrage du serveur Ollama...
✅ Serveur Ollama prêt!

📥 Téléchargement de Gemma 3 4B (≈3GB)...
   Cela peut prendre plusieurs minutes...


✅ Gemma 3 4B téléchargé!
🔥 Préchauffage du modèle...
✅ Gemma 3 4B prêt et opérationnel!

🤖 OLLAMA CONFIGURÉ
Modèle: gemma3:4b
Serveur: http://localhost:11434


## Étape 5 — Configurer Marker
Chargement des modèles Marker (~4GB) depuis le cache Drive.

**⚡ Premier lancement:** 10-20 minutes (téléchargement + sauvegarde sur Drive)

**🚀 Lancements suivants:** 1-2 minutes (chargement depuis Drive)

In [33]:
import json
import re
import shutil
import base64
import logging
import warnings
import os
import sys
import time
from datetime import datetime
from pathlib import Path

# === OPTION: EXTRACTION D'IMAGES ===
EXTRACT_IMAGES = False  # Mettre True pour extraire les images (peut causer des erreurs avec certains PDFs)

# === CONFIGURATION DES LOGS ===
# Désactiver les warnings non critiques
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# Configurer le logging pour réduire la verbosité
logging.getLogger().setLevel(logging.WARNING)

# Réduire drastiquement les logs des bibliothèques tierces
for logger_name in ['transformers', 'torch', 'PIL', 'urllib3', 'filelock',
                     'huggingface_hub', 'marker', 'datasets']:
    logging.getLogger(logger_name).setLevel(logging.ERROR)

# Variables d'environnement pour réduire les logs
os.environ['TRANSFORMERS_VERBOSITY'] = 'error'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['HF_HUB_DISABLE_PROGRESS_BARS'] = '0'  # Garder les barres de progression

# S'assurer que le cache Drive est bien configuré
os.environ["HF_HOME"] = str(HF_CACHE_DRIVE)
os.environ["TRANSFORMERS_CACHE"] = str(HF_CACHE_DRIVE)
os.environ["HUGGINGFACE_HUB_CACHE"] = str(HF_CACHE_DRIVE)
os.environ["TORCH_HOME"] = str(TORCH_CACHE_DRIVE)
# Forcer HF Transfer
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

# === FORCER LE TÉLÉCHARGEMENT DES MODÈLES SURYA DEPUIS HUGGING FACE ===
os.environ["SURYA_LAYOUT_MODEL"] = "vikp/surya_layout"
os.environ["SURYA_REC_MODEL"] = "vikp/surya_rec"
os.environ["SURYA_DET_MODEL"] = "vikp/surya_det"
os.environ["SURYA_ORDER_MODEL"] = "vikp/surya_order"
os.environ["SURYA_TABLE_REC_MODEL"] = "vikp/surya_tablerec"
os.environ["SURYA_DOWNLOAD_BACKEND"] = "huggingface"

import torch
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
print("="*60)
print("🖥️ CONFIGURATION MATÉRIELLE")
print("="*60)
print(f"Device: {device}")
if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# Vérifier le cache existant
def get_dir_size(path):
    total = 0
    if path.exists():
        for f in path.rglob("*"):
            if f.is_file():
                try:
                    total += f.stat().st_size
                except:
                    pass
    return total / 1e9

initial_cache_size = get_dir_size(DRIVE_CACHE_DIR)
print(f"\n💾 Cache Drive: {initial_cache_size:.2f} GB")

if initial_cache_size > 3:
    print("🚀 Modèles trouvés en cache! Chargement rapide...")
else:
    print("📥 Premier téléchargement des modèles Marker (~4GB)")
    print("   ⏱️ Durée estimée: 10-20 minutes")
    print("   💾 Les modèles seront sauvegardés sur Drive")

print("="*60)

# Chronomètre
start_time = time.time()

# Charger les modèles avec messages de progression
print("\n📦 Chargement des modèles Marker...")
if initial_cache_size > 3:
    print("   ⚡ Chargement depuis le cache Drive...")
else:
    print("   📥 Téléchargement en cours...")

from marker.converters.pdf import PdfConverter
from marker.models import create_model_dict
from marker.config.parser import ConfigParser

# Tenter d'importer le service LLM approprié
# On utilise OpenAILLMService pour communiquer avec Ollama
LLM_SERVICE_CLS = None
# NOTE: USE_OLLAMA is read from the global scope here. Ensure it's True if you want LLM with Marker.
# Forcing USE_OLLAMA = True here to align with user's likely intent if Ollama is installed.
USE_OLLAMA = True # Explicitly set to True here for Marker config

if USE_OLLAMA:
    try:
        # Try the most common path first
        from marker.llm.openai import OpenAILLMService
        LLM_SERVICE_CLS = OpenAILLMService
        print("✅ Service LLM: OpenAILLMService (compatible Ollama)")
    except ImportError:
        try:
            # Fallback tentative autre chemin (if Marker has a different structure)
            from marker.services.openai import OpenAILLMService
            LLM_SERVICE_CLS = OpenAILLMService
            print("✅ Service LLM: OpenAILLMService (via path alternatif)")
        except ImportError:
            print("⚠️ Impossible d'importer OpenAILLMService. Le LLM sera désactivé pour Marker.")
            USE_OLLAMA = False # Revert if import fails

# Configuration pour Ollama via API OpenAI
if USE_OLLAMA:
    os.environ["MARKER_LLM_PROVIDER"] = "openai"
    os.environ["OPENAI_BASE_URL"] = "http://localhost:11434/v1"
    os.environ["OPENAI_API_KEY"] = "ollama"
    os.environ["OPENAI_MODEL"] = "gemma3:4b"

# Configuration Marker
marker_config = {
    "workers": 2,
    "extract_images": EXTRACT_IMAGES,
    "images_as_base64": False,
    "use_llm": USE_OLLAMA,
    "llm_provider": "openai" if USE_OLLAMA else None,
    "llm_model": "gemma3:4b" if USE_OLLAMA else None,
    "force_ocr": False,
    "languages": ["fr", "en"],
    "paginate_output": True,
    "batch_size": 4 if device == "cuda" else 2,
}

print("\n⚙️ Création du dictionnaire de modèles...")
model_dict = create_model_dict()

print("✓ Configuration du convertisseur...")
config_parser = ConfigParser(marker_config)

# Instanciation avec le service LLM explicite si disponible
# Relying on environment variables set above (MARKER_LLM_PROVIDER, OPENAI_BASE_URL, etc.)
# for Marker to pick up Ollama LLM config if USE_OLLAMA is True in marker_config.
# Removed explicit llm_service argument here to avoid ImportError blocking other settings.
converter = PdfConverter(
    config=config_parser.generate_config_dict(),
    artifact_dict=model_dict
)

# Résumé
elapsed = time.time() - start_time
final_cache_size = get_dir_size(DRIVE_CACHE_DIR)
downloaded = final_cache_size - initial_cache_size

print("\n" + "="*60)
print("✅ MARKER CONFIGURÉ AVEC SUCCÈS!")
print("="*60)
print(f"⏱️ Durée: {elapsed/60:.1f} minutes")
print(f"📦 Téléchargé cette session: {max(0, downloaded):.2f} GB")
print(f"💾 Cache total sur Drive: {final_cache_size:.2f} GB")
print(f"🖼️ Extraction d'images: {'Activée' if EXTRACT_IMAGES else 'Désactivée'}")
if USE_OLLAMA:
    print(f"🤖 LLM: Gemma 3 4B via Ollama (interface OpenAI)")
else:
    print("🤖 LLM: Désactivé")
print("="*60)
if downloaded > 0.5:
    print("\n💡 Les modèles sont maintenant en cache sur votre Drive.")


🖥️ CONFIGURATION MATÉRIELLE
Device: cuda
GPU: Tesla T4
VRAM: 15.8 GB

💾 Cache Drive: 3.53 GB
🚀 Modèles trouvés en cache! Chargement rapide...

📦 Chargement des modèles Marker...
   ⚡ Chargement depuis le cache Drive...
⚠️ Impossible d'importer OpenAILLMService. Le LLM sera désactivé pour Marker.

⚙️ Création du dictionnaire de modèles...
✓ Configuration du convertisseur...

✅ MARKER CONFIGURÉ AVEC SUCCÈS!
⏱️ Durée: 0.4 minutes
📦 Téléchargé cette session: 0.00 GB
💾 Cache total sur Drive: 3.53 GB
🖼️ Extraction d'images: Désactivée
🤖 LLM: Désactivé


## Étape 6 — Fonctions utilitaires
Extraction des références, figures et conversion PDF → Markdown.

In [36]:
def extract_references_from_markdown(markdown_text):
    """Extrait la section références/bibliographie du Markdown."""
    references = {
        "references_text": "",
        "references_list": [],
        "reference_count": 0,
    }

    ref_patterns = [
        r"(?i)(?:^|\n)#{1,3}\s*(references|références|bibliography|bibliographie|works\s*cited|sources?)\s*[\s: ]*\n([\s\S]*?)(?=\n#{1,3}\s|\Z)",
        r"(?i)(?:^|\n)\*\*(references|références|bibliography|bibliographie)\*\*\s*[\s: ]*\n([\s\S]*?)(?=\n\*\*|\n#{1,3}|\Z)",
    ]

    for pattern in ref_patterns:
        match = re.search(pattern, markdown_text, re.MULTILINE)
        if match:
            ref_section = match.group(2).strip()
            references["references_text"] = ref_section

            ref_lines = []
            lines = ref_section.split("\n")
            current_ref = ""

            for line in lines:
                line = line.strip()
                if not line:
                    if current_ref:
                        ref_lines.append(current_ref. strip())
                        current_ref = ""
                    continue

                if re.match(r"^(\[\d+\]|\d+\.|[-•]|\([A-Z])", line):
                    if current_ref:
                        ref_lines.append(current_ref.strip())
                    current_ref = line
                else:
                    current_ref += " " + line

            if current_ref:
                ref_lines. append(current_ref.strip())

            references["references_list"] = [r for r in ref_lines if len(r) > 20]
            references["reference_count"] = len(references["references_list"])
            break

    return references


def extract_figures_info(markdown_text, images_dict):
    """Extrait les informations sur les figures du document."""
    figures = []
    fig_pattern = r"(?i)(figure|fig\.)\s*(\d+)?\s*[:]? \s*(. {0,120})"

    for match in re.finditer(fig_pattern, markdown_text):
        title = (match.group(3) or "").strip()
        figures.append({
            "label": match.group(0).strip(),
            "title": title,
        })

    if images_dict:
        for img_name in images_dict. keys():
            existing = any(f.get("path") == img_name for f in figures)
            if not existing:
                figures.append({
                    "label": str(img_name),
                    "title": "",
                    "path": str(img_name),
                })

    return figures


def save_figures(images_dict, doc_folders):
    """Sauvegarde les figures extraites dans le dossier _FIGURES du document."""
    if not images_dict:
        return []

    figures_folder = doc_folders["figures"]
    saved_paths = []
    used_names = set()  # Pour éviter les collisions de noms

    for idx, (img_name, img_data) in enumerate(images_dict.items(), 1):
        # Nettoyer le nom du fichier en retirant tous les caractères problématiques
        safe_name = re.sub(r"[^a-zA-Z0-9_-]+", "_", str(img_name))
        # Supprimer les underscores multiples et en début/fin
        safe_name = re.sub(r"_+", "_", safe_name).strip("_")
        # Limiter la longueur du nom (max 100 caractères)
        safe_name = safe_name[:100]
        # Si le nom est vide après nettoyage, utiliser un index
        if not safe_name or len(safe_name) < 3:
            safe_name = f"figure_{idx:03d}"

        # Gérer les collisions de noms
        base_name = safe_name
        counter = 1
        while safe_name in used_names:
            safe_name = f"{base_name}_{counter}"
            counter += 1
        used_names.add(safe_name)

        img_path = figures_folder / f"{safe_name}.png"

        try:
            if isinstance(img_data, Image.Image):
                # Convertir en RGB si nécessaire (pour les images RGBA/P)
                if img_data.mode in ('RGBA', 'LA', 'P'):
                    rgb_image = Image.new('RGB', img_data.size, (255, 255, 255))
                    if img_data.mode == 'P':
                        img_data = img_data.convert('RGBA')
                    rgb_image.paste(img_data, mask=img_data.split()[-1] if img_data.mode in ('RGBA', 'LA') else None)
                    img_data = rgb_image
                img_data.save(img_path, "PNG", optimize=True)
            elif isinstance(img_data, (bytes, bytearray)):
                # Sauvegarder directement les bytes
                with open(img_path, "wb") as f:
                    f.write(img_data)
            elif isinstance(img_data, str):
                if img_data.startswith("data:image"):
                    # Données base64 encodées
                    b64_data = img_data.split(",", 1)[1] if "," in img_data else img_data
                    with open(img_path, "wb") as f:
                        f.write(base64.b64decode(b64_data))
                elif Path(img_data).exists():
                    # Copie depuis un fichier existant
                    shutil.copy(img_data, img_path)
                else:
                    # Tenter de décoder comme base64
                    try:
                        with open(img_path, "wb") as f:
                            f.write(base64.b64decode(img_data))
                    except:
                        continue
            else:
                continue

            saved_paths.append(str(img_path))
        except Exception as e:
            # Log l'erreur mais continue le traitement
            print(f"   ⚠️ Impossible de sauvegarder {safe_name}: {str(e)[:80]}")
            continue

    return saved_paths


def convert_pdf_complete(pdf_path, doc_name):
    """Conversion complète d'un PDF avec extraction figures et références."""
    doc_folders = get_doc_folders(doc_name)

    result_data = {
        "doc_name": doc_name,
        "doc_folder": str(doc_folders["root"]),
        "markdown_path": "",
        "figures": [],
        "figures_paths": [],
        "references": {},
        "error": None,
    }

    try:
        print(f"   DEBUG: Tentative de conversion initiale pour {pdf_path.name}...")
        # Conversion Marker avec gestion d'erreur robuste
        try:
            result = converter(str(pdf_path))
            print("   DEBUG: Première conversion réussie.")
        except Exception as conv_error:
            import traceback # Ensure traceback is available
            print(f"   DEBUG: Erreur capturée dans première tentative: {conv_error}")
            traceback.print_exc() # Print full traceback here

            # If the error concerns images or is of type "unknown extension", try without image extraction
            if "extension" in str(conv_error).lower() or "image" in str(conv_error).lower() or "unknown extension" in str(conv_error).lower():
                print(f"   ⚠️ Erreur d'extraction d'images (première tentative): {str(conv_error)[:60]}")
                print("   🔄 Nouvelle tentative SANS extraction d'images...")

                # Reconfigure temporarily Marker without image extraction
                from marker.config.parser import ConfigParser
                temp_config = {
                    "workers": 2,
                    "extract_images": False,  # Disable image extraction
                    "images_as_base64": False,
                    "use_llm": False, # LLM disabled for fallback attempt
                    "force_ocr": False,
                    "languages": ["fr", "en"],
                    "paginate_output": True,
                    "batch_size": 4 if device == "cuda" else 2,
                }
                config_parser = ConfigParser(temp_config)
                from marker.converters.pdf import PdfConverter
                temp_converter = PdfConverter(
                    config=config_parser.generate_config_dict(),
                    artifact_dict=model_dict,
                    llm_service=None # Ensure no LLM service for this fallback
                )
                try:
                    result = temp_converter(str(pdf_path))
                    print(f"   DEBUG: Nouvelle tentative réussie sans extraction d'images.")
                except Exception as second_conv_error:
                    print(f"   ❌ Erreur (seconde tentative sans images): {second_conv_error}")
                    traceback.print_exc() # Print full traceback here for second attempt
                    raise second_conv_error # Relaunch the error if the second attempt also fails
            else:
                raise # Relaunch the error if it's not an image/extension issue

        markdown_text = getattr(result, "markdown", "") or ""
        images_dict = getattr(result, "images", {}) or {}

        # Nettoyer le dictionnaire d'images pour enlever les entrées invalides
        cleaned_images = {}
        if images_dict:
            for key, value in images_dict.items():
                # Ignorer les clés ou valeurs None/vides
                if key and value is not None:
                    # Vérifier que la clé ne contient pas de caractères problématiques
                    if not any(char in str(key) for char in ['?', '*', '<', '>', '|', '\x00']):
                        cleaned_images[key] = value
                    else:
                        print(f"   ⚠️ Image ignorée (nom invalide): {str(key)[:30]}")

        images_dict = cleaned_images

        md_path = doc_folders["root"] / f"{doc_name}.md"
        with open(md_path, "w", encoding="utf-8") as f:
            f.write(markdown_text)

        result_data["markdown_path"] = str(md_path)
        result_data["figures_paths"] = save_figures(images_dict, doc_folders)
        result_data["figures"] = extract_figures_info(markdown_text, images_dict)

        result_data["references"] = extract_references_from_markdown(markdown_text)
        if result_data["references"].get("reference_count", 0) > 0:
            ref_path = doc_folders["references"] / f"{doc_name}_references.json"
            with open(ref_path, "w", encoding="utf-8") as f:
                json.dump(result_data["references"], f, ensure_ascii=False, indent=2)

        return result_data
    except Exception as e:
        import traceback # Ensure traceback is imported
        print(f"   DEBUG: Capture d'erreur générique dans convert_pdf_complete: {e}")
        traceback.print_exc() # Print the full traceback for outer catch
        result_data["error"] = str(e)
        error_log = doc_folders["logs"] / "error.txt"
        with open(error_log, "w", encoding="utf-8") as f:
            f.write(f"Erreur: {str(e)}\nTraceback:\n{traceback.format_exc()}\nDate: {datetime.now().isoformat()}")
        return result_data

print("✅ Fonctions utilitaires définies. ")

✅ Fonctions utilitaires définies. 


## Étape 7 — LangExtract
Extraction structurée avec LangExtract (activé par défaut avec Ollama).

In [19]:
USE_LANGEXTRACT = True

# Vérifier si Ollama est disponible (indépendamment de la config Marker)
OLLAMA_AVAILABLE = False
try:
    import requests
    response = requests.get("http://localhost:11434/api/tags", timeout=2)
    if response.status_code == 200:
        OLLAMA_AVAILABLE = True
        print("✅ Ollama détecté - Utilisation de Gemma 3 4B")
except:
    print("⚠️ Ollama non disponible - Utilisation de OpenAI requis")

LANGEXTRACT_CONFIG = {
    "provider": "ollama" if OLLAMA_AVAILABLE else "openai",
    "model": "gemma3:4b" if OLLAMA_AVAILABLE else "gpt-3.5-turbo",
    "base_url": "http://localhost:11434" if OLLAMA_AVAILABLE else None,
}

PROMPT_TEMPLATE = """Vous êtes un assistant d'analyse pour des documents en sciences sociales.
Retournez un JSON structuré avec les sections suivantes:

1.  CONTEXTE
- Thème principal
- Zone géographique
- Période

2. ACTEURS
- Institutions
- Pays
- Organisations

3. CONCEPTS CLÉS
- Mots-clés
- Concepts

4. DONNÉES
- Chiffres clés (si disponibles)

5. RÉFÉRENCES
- Principales références citées

6. FIGURES ET TABLEAUX
- Liste des figures mentionnées

Répondez uniquement avec un JSON valide."""

def _safe_json(obj):
    try:
        return json.loads(json.dumps(obj))
    except Exception:
        return {"raw": str(obj)}


def extract_with_langextract(markdown_text, doc_name, doc_folders, references_data=None, figures_data=None):
    """Extraction structurée avec LangExtract."""
    if not USE_LANGEXTRACT:
        return {"status": "skipped", "reason": "USE_LANGEXTRACT=False"}

    enriched_text = markdown_text

    if references_data and references_data.get("reference_count", 0) > 0:
        enriched_text += "\n\n## RÉFÉRENCES\n"
        enriched_text += f"Nombre de références: {references_data['reference_count']}\n"
        for i, ref in enumerate(references_data. get("references_list", [])[:20], 1):
            enriched_text += f"[{i}] {ref}\n"

    if figures_data:
        enriched_text += "\n\n## FIGURES IDENTIFIÉES\n"
        enriched_text += f"Nombre de figures:  {len(figures_data)}\n"
        for fig in figures_data[: 10]:
            enriched_text += f"- {fig. get('label', '')} {fig.get('title', '')}\n"

    try:
        import langextract as lx

        extractor_kwargs = {
            "prompt": PROMPT_TEMPLATE,
        }

        if OLLAMA_AVAILABLE and LANGEXTRACT_CONFIG["base_url"]:
            extractor_kwargs["provider"] = "ollama"
            extractor_kwargs["model"] = LANGEXTRACT_CONFIG["model"]
            extractor_kwargs["base_url"] = LANGEXTRACT_CONFIG["base_url"]

        if hasattr(lx, "extract"):
            extraction = lx.extract(enriched_text, **extractor_kwargs)
        elif hasattr(lx, "LangExtract"):
            extractor = lx.LangExtract(**extractor_kwargs)
            extraction = extractor. extract(enriched_text)
        else:
            return {"status": "error", "error": "API LangExtract introuvable"}

        result = _safe_json(extraction)

        extraction_path = doc_folders["analyses"] / f"{doc_name}_langextract.json"
        with open(extraction_path, "w", encoding="utf-8") as f:
            json.dump(result, f, ensure_ascii=False, indent=2)

        return result
    except Exception as e:
        return {"status":  "error", "error":  str(e)}

print(f"✅ LangExtract configuré (Provider: {LANGEXTRACT_CONFIG['provider']}, Model: {LANGEXTRACT_CONFIG['model']})")

✅ Ollama détecté - Utilisation de Gemma 3 4B
✅ LangExtract configuré (Provider: ollama, Model: gemma3:4b)


## Étape 8 — Test sur un PDF (optionnel)
Permet de valider la configuration avant le batch.

In [37]:
pdf_files = sorted([p for p in INPUT_DIR.iterdir() if p.suffix.lower() == ".pdf"])

if pdf_files:
    sample_path = pdf_files[0]
    sample_name = sample_path.stem
    print(f"🧪 Test sur:  {sample_path.name}")
    print("⏳ Conversion en cours...\n")

    test_start = time.time()
    sample_result = convert_pdf_complete(sample_path, sample_name)
    test_elapsed = time.time() - test_start

    if sample_result.get("error"):
        print(f"❌ Erreur:  {sample_result['error']}")
    else:
        print(f"✅ Conversion réussie en {test_elapsed:.1f}s! ")
        print(f"   📂 Dossier: {sample_result['doc_folder']}")
        print(f"   📄 Markdown: {sample_result['markdown_path']}")
        print(f"   🖼️ Figures: {len(sample_result['figures_paths'])}")
        print(f"   📚 Références: {sample_result['references'].get('reference_count', 0)}")
else:
    print("❌ Aucun PDF trouvé dans le dossier d'entrée.")

🧪 Test sur:  Against Sovereignty in Cyberspace.pdf
⏳ Conversion en cours...

   DEBUG: Tentative de conversion initiale pour Against Sovereignty in Cyberspace.pdf...


Recognizing tables: 100%|██████████| 1/1 [00:00<00:00,  3.93it/s]
Detecting bboxes: 0it [00:00, ?it/s]


   DEBUG: Première conversion réussie.
✅ Conversion réussie en 104.1s! 
   📂 Dossier: /content/drive/MyDrive/Géopolitique et Souveraineté Numériques/ALL/ALLMD/Against Sovereignty in Cyberspace
   📄 Markdown: /content/drive/MyDrive/Géopolitique et Souveraineté Numériques/ALL/ALLMD/Against Sovereignty in Cyberspace/Against Sovereignty in Cyberspace.md
   🖼️ Figures: 0
   📚 Références: 0


## Étape 9 — Pipeline complet avec reprise
Traitement batch + logs + reprise automatique.

In [ ]:
import time as time_module

def process_all_documents():
    """Traite tous les documents PDF avec reprise automatique."""

    all_pdfs = sorted([p for p in INPUT_DIR.iterdir() if p.suffix.lower() == ".pdf"])

    if not all_pdfs:
        print("❌ Aucun PDF trouvé dans le dossier d'entrée! ")
        print(f"   Chemin vérifié: {INPUT_DIR}")
        return [], []

    log_file = GLOBAL_LOGS_DIR / f"processing_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
    progress_file = GLOBAL_LOGS_DIR / "progress.json"

    def log_message(message):
        print(message)
        with open(log_file, "a", encoding="utf-8") as f:
            f.write(message + "\n")

    processed_files = set()
    if progress_file.exists():
        try:
            with open(progress_file, "r", encoding="utf-8") as f:
                progress_data = json.load(f)
                processed_files = set(progress_data.get("processed", []))
            log_message(f"📂 Reprise:  {len(processed_files)} fichiers déjà traités")
        except Exception:
            processed_files = set()

    remaining_pdfs = [p for p in all_pdfs if p. name not in processed_files]

    log_message("="*60)
    log_message(f"📄 Total PDFs: {len(all_pdfs)}")
    log_message(f"✅ Déjà traités: {len(processed_files)}")
    log_message(f"⏳ Restants: {len(remaining_pdfs)}")
    log_message("="*60)

    if not remaining_pdfs:
        log_message("✅ Tous les fichiers ont déjà été traités!")
        return [], []

    results = []
    errors = []
    start_time = time_module.time()

    def save_progress():
        with open(progress_file, "w", encoding="utf-8") as f:
            json.dump({
                "processed": list(processed_files),
                "last_update": datetime.now().isoformat()
            }, f, ensure_ascii=False, indent=2)

    for idx, pdf_path in enumerate(remaining_pdfs, 1):
        doc_name = pdf_path.stem
        doc_start_time = time_module.time()

        log_message(f"\n[{idx}/{len(remaining_pdfs)}] 🚀 Traitement:  {pdf_path.name}")

        conversion_result = convert_pdf_complete(pdf_path, doc_name)

        if conversion_result.get("error"):
            errors. append({"file": pdf_path. name, "error": conversion_result["error"]})
            log_message(f"   ❌ Erreur conversion: {conversion_result['error']}")
            processed_files.add(pdf_path.name)
            save_progress()
            continue

        doc_folders = get_doc_folders(doc_name)

        log_message(
            f"   🖼️ Figures: {len(conversion_result['figures'])} trouvées, "
            f"{len(conversion_result['figures_paths'])} sauvegardées"
        )
        log_message(
            f"   📚 Références: {conversion_result['references'].get('reference_count', 0)} extraites"
        )

        extraction = None
        if USE_LANGEXTRACT:
            log_message("   🔍 Extraction LangExtract... ")
            with open(conversion_result['markdown_path'], 'r', encoding='utf-8') as f:
                md_content = f.read()
            extraction = extract_with_langextract(
                md_content,
                doc_name,
                doc_folders,
                conversion_result['references'],
                conversion_result['figures'],
            )
            if extraction. get("status") == "error":
                log_message(f"   ⚠️ LangExtract: {extraction. get('error', 'Erreur inconnue')}")
            else:
                log_message("   ✅ LangExtract terminé")

        analysis = {
            "doc_name": doc_name,
            "source_pdf": str(pdf_path),
            "doc_folder": str(doc_folders["root"]),
            "processed_at": datetime.now().isoformat(),
            "processing_time_seconds": round(time_module.time() - doc_start_time, 2),
            "conversion": {
                "markdown_path": conversion_result['markdown_path'],
                "figures_count": len(conversion_result['figures']),
                "figures_saved": len(conversion_result['figures_paths']),
                "figures_paths": conversion_result['figures_paths'],
                "references_count": conversion_result['references'].get('reference_count', 0),
            },
            "references": conversion_result['references'],
            "figures": conversion_result['figures'],
            "langextract": extraction,
        }

        analysis_path = doc_folders["analyses"] / f"{doc_name}_analysis.json"
        with open(analysis_path, "w", encoding="utf-8") as f:
            json.dump(analysis, f, ensure_ascii=False, indent=2)

        doc_log_path = doc_folders["logs"] / "processing. log"
        with open(doc_log_path, "w", encoding="utf-8") as f:
            f. write(f"Traité le: {datetime.now().isoformat()}\n")
            f.write(f"Durée: {analysis['processing_time_seconds']} secondes\n")
            f.write(f"Figures:  {len(conversion_result['figures_paths'])}\n")
            f.write(f"Références: {conversion_result['references'].get('reference_count', 0)}\n")

        results.append(analysis)
        processed_files.add(pdf_path.name)
        save_progress()

        elapsed = time_module.time() - doc_start_time
        log_message(f"   ⏱️ Durée: {elapsed:.1f}s")

    total_time = time_module.time() - start_time
    log_message("\n" + "="*60)
    log_message(f"✅ Traitement terminé en {total_time:.1f} secondes")
    log_message(f"   📄 Réussis: {len(results)}")
    log_message(f"   ❌ Erreurs: {len(errors)}")
    log_message("="*60)

    return results, errors


def export_all_results(results):
    """Exporte les résultats globaux."""
    if not results:
        print("ℹ️ Aucun résultat à exporter. ")
        return

    summary_path = GLOBAL_LOGS_DIR / "_SUMMARY.json"
    report_md_path = GLOBAL_LOGS_DIR / "_REPORT. md"

    with open(summary_path, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)

    all_references = []
    for data in results:
        refs = data.get("references", {})
        if refs. get("references_list"):
            for ref in refs["references_list"]:
                all_references.append({
                    "document": data. get("doc_name", ""),
                    "reference": ref,
                })

    if all_references:
        biblio_path = GLOBAL_LOGS_DIR / "_BIBLIOGRAPHIE_COMPLETE.json"
        with open(biblio_path, "w", encoding="utf-8") as f:
            json. dump(all_references, f, ensure_ascii=False, indent=2)
        print(f"📚 Bibliographie:  {biblio_path} ({len(all_references)} références)")

    all_figures = []
    for data in results:
        for fig_path in data.get("conversion", {}).get("figures_paths", []):
            all_figures.append({
                "document": data. get("doc_name", ""),
                "path": fig_path,
            })

    if all_figures:
        figures_index_path = GLOBAL_LOGS_DIR / "_INDEX_FIGURES.json"
        with open(figures_index_path, "w", encoding="utf-8") as f:
            json.dump(all_figures, f, ensure_ascii=False, indent=2)
        print(f"🖼️ Index figures: {figures_index_path} ({len(all_figures)} figures)")

    md_lines = []
    md_lines.append("# Rapport de traitement MFEGSN\n\n")
    md_lines.append(f"**Date:** {datetime.now().strftime('%Y-%m-%d %H:%M')}\n\n")
    md_lines.append(f"- **Total documents traités:** {len(results)}\n")
    md_lines.append(f"- **Total figures extraites:** {len(all_figures)}\n")
    md_lines.append(f"- **Total références:** {len(all_references)}\n\n")
    md_lines.append("## Documents traités\n\n")

    for data in results:
        conv = data.get("conversion", {})
        md_lines.append(f"### {data. get('doc_name', '')}\n\n")
        md_lines.append(f"- **Dossier:** `{data.get('doc_folder', '')}`\n")
        md_lines.append(f"- **Figures:** {conv.get('figures_count', 0)}\n")
        md_lines. append(f"- **Références:** {conv.get('references_count', 0)}\n")
        md_lines.append(f"- **Durée:** {data.get('processing_time_seconds', 0)}s\n\n")

    with open(report_md_path, "w", encoding="utf-8") as f:
        f.write("". join(md_lines))

    print(f"✅ Résultats exportés: ")
    print(f"   📊 Summary: {summary_path}")
    print(f"   📝 Report: {report_md_path}")

print("✅ Fonctions de pipeline définies. ")

## Étape 10 — Lancer le traitement
Exécutez cette cellule pour lancer le traitement complet.

In [ ]:
# Recharger la liste des PDFs depuis INPUT_DIR pour s'assurer qu'elle est à jour
print("📂 Vérification du dossier d'entrée...")
print(f"   Chemin: {INPUT_DIR}")

# Recharger la liste des fichiers
pdf_files = sorted([p for p in INPUT_DIR.iterdir() if p.suffix.lower() == ".pdf"])
print(f"   PDFs trouvés: {len(pdf_files)}")

if len(pdf_files) == 0:
    print("\n⚠️ ATTENTION: Aucun PDF trouvé!")
    print("   Vérifiez que:")
    print("   1. Le chemin INPUT_DIR est correct")
    print("   2. Des fichiers PDF sont présents dans ce dossier")
    print("   3. Google Drive est bien monté")
else:
    for i, pdf in enumerate(pdf_files[:5], 1):
        print(f"   {i}. {pdf.name}")
    if len(pdf_files) > 5:
        print(f"   ... et {len(pdf_files) - 5} autres")

print("\n" + "="*60)
print("🚀 Démarrage du pipeline...")
print("="*60)

results, errors = process_all_documents()

if results:
    export_all_results(results)

print("\n" + "="*60)
print("✅ PIPELINE TERMINÉ")
print("="*60)

if errors:
    print(f"\n⚠️ {len(errors)} erreur(s) rencontrée(s):")
    for err in errors[:5]:
        print(f"   - {err['file']}: {err['error'][:50]}...")
    if len(errors) > 5:
        print(f"   ... et {len(errors) - 5} autres erreurs")

## Utilitaires — Gestion du cache
Cellules optionnelles pour gérer le cache des modèles sur Drive.

In [ ]:
# === AFFICHER LA TAILLE DU CACHE ===
def show_cache_info():
    """Affiche les informations sur le cache Drive."""
    print("="*60)
    print("💾 INFORMATIONS CACHE")
    print("="*60)

    for name, path in [("HuggingFace", HF_CACHE_DRIVE),
                       ("Torch", TORCH_CACHE_DRIVE),
                       ("Datalab", DATALAB_CACHE_DRIVE)]:
        size = get_dir_size(path)
        print(f"📂 {name}: {size:.2f} GB")

    total = get_dir_size(DRIVE_CACHE_DIR)
    print(f"\n📦 Total: {total:.2f} GB")
    print(f"📍 Emplacement: {DRIVE_CACHE_DIR}")
    print("="*60)

# Appeler la fonction pour afficher les infos
show_cache_info()

In [ ]:
# === SUPPRIMER LE CACHE (si nécessaire) ===
# ⚠️ ATTENTION:  Cela supprimera tous les modèles en cache!
# Décommentez les lignes ci-dessous pour exécuter.

# import shutil
#
# if DRIVE_CACHE_DIR. exists():
#     print(f"⚠️ Suppression du cache:  {DRIVE_CACHE_DIR}")
#     shutil.rmtree(DRIVE_CACHE_DIR)
#     print("✅ Cache supprimé.  Les modèles seront retéléchargés au prochain lancement.")
# else:
#     print("ℹ️ Aucun cache à supprimer.")

print("ℹ️ Décommentez le code ci-dessus pour supprimer le cache.")